# Release of 100 Particles in each Ocean Cell of BOATS using OceansParcels

Sandra Neubert
14/12/2022

## Background

The overall goal is to inlcude larval movement in the ecological module of BOATS. To this end, we explore an appraoch based on [van Sebille (2011)](https://doi.org/10.1175/2011JPO4602.1) to create transition matrices giving the probability of moving from a $cell_{i,j}$ to the eight directly adjacent cells. Since actual (drifter) data is not available for all parts of the ocean, we explore an alternative approach that uses particle tracking simulations to generate simulated data for later probability calculations. We will use the Python package OceanParcels ([Delandmeter and van Sebille, 2019](https://www.geosci-model-dev.net/12/3571/2019/gmd-12-3571-2019.html)) to track particles.

### Setup 

***Important comment:*** <br/>
Error for netcdf4 package unless right combination of Python (3.9) and netcdf4 (1.5.7) is used (DLL not found, Anaconda venv issue with windows). To create a working venv, run:

`conda create -n yourEnvName python=3.9`

`conda activate yourEnvName #here: py3_parcels3`

`conda install -c conda-forge parcels jupyter spyder cartopy ffmpeg netcdf4=1.5.7`

To test if netcdf4 is working run (in Jupyter or Spyder or with python):

`from netCDF4 import Dataset`

### 1. Load Packages

In [ ]:
import os
import numpy as np
import xarray as xr
import geopandas as gp
import pandas as pd

from parcels import FieldSet, Field, ParticleSet, Variable, JITParticle
from parcels import AdvectionRK4, plotTrajectoriesFile, ErrorCode

import math
from datetime import timedelta as delta
from operator import attrgetter

### 2. Data

### 2.1 Hydrodynamic Model Data

[OFES data](http://apdrc.soest.hawaii.edu/erddap/search/index.html?page=1&itemsPerPage=1000&searchFor=OfES+ncep+0.5+global+mmean), specifically [zonal velocity (u)](http://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_6c0d_24b8_6937.html) and [meridional velocity (v)](http://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_c66b_2477_13c0.html) can be downloaded by selecting the desired intervals (here: 2015-01-15 to 2019-12-15T00:00:00, for 2.5-2.5 m (one layer) LEV), and then *submitting* after selecting *.nc* as the file type (or .csv when .nc is too big).


In [ ]:
dataPath = "C:\\Users\\sandr\\Documents\\Github\ThesisSandra\\Analysis\\Movement\\TracerDataAndOutput\\OFES\\"
ufiles = dataPath + "OfESncep01globalmmeanu20152019MS.nc"
vfiles = dataPath + "OfESncep01globalmmeanv20152019MS.nc"
filenames = {'U': ufiles,
             'V': vfiles}

variables = {'U': 'u',
             'V': 'v'}
dimensions = {'lat': 'latitude',
              'lon': 'longitude',
              'time': 'time'}

### 2.2 BOATS Data

Ultimately, we aim to release 100 particles from each grid cell of BOATS monthly for the temporal extent of the OFES data and then calculate monthly (12) probability matrices. Start locations were determined by using the spatial extent of the OFES data which will subsequently be mapped onto the BOATS grid.

In [ ]:
StartLocations = pd.read_csv('C:\\Users\\sandr\\Documents\\Github\\ThesisSandra\\Analysis\\Movement\\Data\\dfOFESStartLocationsGlobal2.csv')
StartLocations = df[['lon','lat']]

### 3. Define Field Set

In [ ]:
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)
fieldset.add_constant('maxage', 3.*86400) #get rid of particles after 3 days
fieldset.add_periodic_halo(zonal=True) #to not get artifacts around prime meridian (linked to kernel further down)

### 4. Define Simulation Conditions

In [ ]:
lon_array = StartLocations.lon
lat_array = StartLocations.lat

npart = 1 #how many particles are released at each location (every time)
lon = np.repeat(lon_array, npart)
lat = np.repeat(lat_array, npart)

In [ ]:
# How often to release the particles; 
#Probelm: if I release particles over a long period of time, setting the repeatdt to 30 days leads to particles being released on a different day each month and it gets worse with time
#if I set repeatdt at 30.4375, release dates stay around the same
repeatdt = delta(days = 30.4375) # release from the same set of locations every months

start_time = datetime(2015,1,15)
end_time = datetime(2019,11,15)  #year, month, day,

runtime = end_time-start_time + delta(days=15) #add some days at the end to make sure tracking can be done for 5 days from the last start location onwards if release date is not exactly on 15th

time = 0 #np.arange(0, npart) * delta(days = 30.4375).total_seconds() 
time

### 5. Define Particle Properties

**Add Beaching kernel**

In [ ]:
class SampleParticle(JITParticle):         # Define a new particle class
        sampled = Variable('sampled', dtype = np.float32, initial = 0, to_write=False)
        age = Variable('age', dtype=np.float32, initial=0.) # initialise age
        distance = Variable('distance', initial=0., dtype=np.float32)  # the distance travelled
        prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
                            initial=0)  # the previous longitude
        prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
                            initial=0)  # the previous latitude
        #beached = Variable('beached', dtype = np.float32, initial = 0)
    
def DeleteParticle(particle, fieldset, time): #needed to avoid error mesasage of Particle out of bounds
    particle.delete()
    
# Define all the sampling kernels
def SampleDistance(particle, fieldset, time):
    # Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    # Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))
    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat

def SampleAge(particle, fieldset, time):
    particle.age = particle.age + math.fabs(particle.dt)
    if particle.age >= fieldset.maxage: #if not >= : get one more particle tracking point after maxage
           particle.delete()
            
def periodicBC(particle, fieldset, time):
    if particle.lon < 0:
        particle.lon += 360 - 0
    elif particle.lon > 359.9:
        particle.lon -= 360 - 0

# def Unbeaching(particle, fieldset, time):
# #     if particle.age == 0 and particle.u_vel == 0 and particle.v_vel == 0: # velocity = 0 means particle is on land so nudge it eastward
# #         particle.lon += random.uniform(0.5, 1) #dont need this because I know my particles dont start on land?
#     if particle.u_vel == 0 and particle.v_vel == 0: # if a particle is advected on to land so mark it as beached (=1)
#         particle.beached = 1
    
def SampleInitial(particle, fieldset, time): # do we have to add particle.age and particle.ageRise
        if particle.sampled == 0:
            particle.distance = particle.distance
            particle.prev_lon = particle.lon
            particle.prev_lat = particle.lat
            #particle.beached = particle.beached
            particle.sampled = 1
               
pset = ParticleSet.from_list(fieldset, 
                             pclass=SampleParticle, 
                             time=time, 
                             lon=lon, 
                             lat=lat,
                             repeatdt=repeatdt)


### 6. Define kernels

In [ ]:
kernels = SampleInitial + pset.Kernel(AdvectionRK4) + periodicBC + SampleAge + SampleDistance

### 7. Execute Particle Tracking

In [ ]:
output_nc_dist = 'NearGlobalParticleTrackingOFES.zarr'
try:
    os.remove(output_nc_dist)
except OSError:
    pass

file_dist = pset.ParticleFile(name=output_nc_dist, 
                                outputdt=delta(hours=6)) #save location every 6 hours

pset.execute(kernels,  
             runtime=runtime,
             dt=delta(minutes=10), #to reduce computational load
             output_file=file_dist,
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:
dfParcels = output_nc_dist.to_dataframe()
dfParcels.to_csv(localPath + 'dfParcelsGlobal.csv')